In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test)= fashion_mnist.load_data()


In [4]:
x_train, y_train = x_train[6000:], y_train[6000:]
x_valid, y_valid = x_train[:6000], y_train[:6000]
len_train = len(x_train)
len_valid = len(x_valid)
print(len_train,len_valid)

54000 6000


In [5]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(images, augment=True, seed=seed)
mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    'data/images',
    class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'data/masks',
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50)

In [6]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)

train_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,**data_gen_args)
train_generator.fit(x_train,augment=True,)
train_gen = train_generator.flow(x_train,y_train,
                     shuffle=True,
                    )

test_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator.fit(x_test)
test_gen = test_generator.flow(x_test,y_test)

valid_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator.fit(x_valid)
valid_gen = valid_generator.flow(x_valid,y_valid)
                    

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, padding='same', input_shape=(28,28,1), activation = 'relu'),
    keras.layers.Conv2D(32, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(64, 3, padding='same',  activation = 'relu'),
    keras.layers.Conv2D(64, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
#     keras.layers.Conv2D(128, 3, padding='same', activation = 'relu'),
#     keras.layers.Conv2D(128, 3,padding='same', activation = 'relu'),
#     keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
    ])
model.compile(loss =keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     optimizer = 'adam',
                     metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=0.01,)]
history = model.fit(train_gen, epochs = 5, steps_per_epoch=54000//32,validation_data = valid_gen,validation_steps = 6000//32)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1687 steps, validate for 187 steps
Epoch 1/5
1687/1687 [==============================] - 121s 72ms/step - loss: 2.3028 - accuracy: 0.1001 - val_loss: 2.3030 - val_accuracy: 0.0968
Epoch 2/5
1687/1687 [==============================] - 120s 71ms/step - loss: 2.3028 - accuracy: 0.0973 - val_loss: 2.3024 - val_accuracy: 0.0854
Epoch 3/5
 299/1687 [====>.........................] - ETA: 1:35 - loss: 2.3027 - accuracy: 0.1031

In [11]:
model.evaluate(test_gen,verbose=2)

  ...
    to  
  ['...']
313/313 - 6s - loss: 0.2256 - accuracy: 0.9235


[0.22563229538143253, 0.9235]

In [15]:
pd.DataFrame(history.history).polt()

NameError: name 'history' is not defined